In [35]:
import tensorflow
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [36]:
def make_one_hot(data,field):
    one_hot=pd.get_dummies(data[field],prefix=field)
    one_hotsum = one_hot.apply(np.sum, axis=1)
    for i in range(data.shape[0]):
        if one_hotsum.iloc[i]==0:
            for j in range(one_hot.shape[1]):
                one_hot.iat[i,j]=np.nan
    data=data.drop(field,axis=1)
    return data.join(one_hot)

In [37]:
df=pd.read_csv("CarPrice.csv")
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [38]:
df=df.drop(["CarName","wheelbase","car_ID","enginelocation"],axis=1)
df=df.drop(["carlength","carwidth","carheight"],axis=1)


df=make_one_hot(df,"carbody")
df=make_one_hot(df,"drivewheel")
df=make_one_hot(df,"enginetype")
df=make_one_hot(df,"fuelsystem")


df.fueltype.replace({"gas":0,"diesel":1},inplace=True)
df.aspiration.replace({"std":0,"turbo":1},inplace=True)
df.doornumber.replace({"two":0,"four":1},inplace=True)
df.cylindernumber.replace({"four":4,"six":6,"five":5,"twelve":12,"three":3,"eight":8,"two":2},inplace=True)
df.symboling.replace({3:1,2:2,1:3,0:4,-1:5,-2:6},inplace=True)

In [39]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

y=df["price"]
x=df.drop("price",axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1)


SS=StandardScaler()
SS.fit(x_train)
x_train=SS.transform(x_train)
x_test=SS.transform(x_test)

In [40]:
model=keras.models.Sequential()
model.add(keras.layers.Dense(100,activation="relu"))
model.add(keras.layers.Dense(50,activation="relu"))
model.add(keras.layers.Dense(1,activation="relu"))

In [41]:
model.compile(metrics="mean_absolute_error",
              loss="mean_absolute_error",
              optimizer="sgd")

In [42]:
history=model.fit(x_train,y_train,epochs=100,validation_split=0.15,callbacks=keras.callback.EarlyS)

Epoch 1/100
5/5 [==============================] - 2s 149ms/step - loss: 13375.2129 - mean_absolute_error: 13375.2129 - val_loss: 13891.4863 - val_mean_absolute_error: 13891.4863
Epoch 2/100
5/5 [==============================] - 0s 20ms/step - loss: 13374.8906 - mean_absolute_error: 13374.8906 - val_loss: 13891.0098 - val_mean_absolute_error: 13891.0098
Epoch 3/100
5/5 [==============================] - 0s 20ms/step - loss: 13374.3682 - mean_absolute_error: 13374.3682 - val_loss: 13890.3623 - val_mean_absolute_error: 13890.3623
Epoch 4/100
5/5 [==============================] - 0s 19ms/step - loss: 13373.7031 - mean_absolute_error: 13373.7031 - val_loss: 13889.5557 - val_mean_absolute_error: 13889.5557
Epoch 5/100
5/5 [==============================] - 0s 24ms/step - loss: 13372.8555 - mean_absolute_error: 13372.8555 - val_loss: 13888.5010 - val_mean_absolute_error: 13888.5010
Epoch 6/100
5/5 [==============================] - 0s 29ms/step - loss: 13371.7383 - mean_absolute_error: 133

In [33]:
model.evaluate(x_test,y_test)

1/1 [==============================] - 0s 34ms/step - loss: 2747.2261 - mean_absolute_error: 2747.2261


[2747.22607421875, 2747.22607421875]